In [6]:
import os
import sys
from PIL import Image

import numpy as np

import keras
from keras.utils import plot_model

from keras_frcnn import models


In [2]:
model = models.FRCNN(keras.Input(shape=(1333, 1333, 3)))

In [4]:
print(model.get_layer('P3').name, model.get_layer('P3').output_shape)
print(model.get_layer('P4').name, model.get_layer('P4').output_shape)
print(model.get_layer('P5').name, model.get_layer('P5').output_shape)
print(model.get_layer('P6').name, model.get_layer('P6').output_shape)
print(model.get_layer('P7').name, model.get_layer('P7').output_shape)
print(model.layers[-1].name, model.layers[-1].output_shape)

P3 (None, 164, 164, 256)
P4 (None, 81, 81, 256)
P5 (None, 40, 40, 256)
P6 (None, 20, 20, 256)
P7 (None, 10, 10, 256)
anchors (None, 320013, 4)


In [5]:
print(model.get_layer('P3').output_shape[1] *   8)
print(model.get_layer('P4').output_shape[1] *  16)
print(model.get_layer('P5').output_shape[1] *  32)
print(model.get_layer('P6').output_shape[1] *  64)
print(model.get_layer('P7').output_shape[1] * 128)


1312
1296
1280
1280
1280


In [8]:
x = np.zeros((1, 1333, 1333, 3))
y = model.predict(x)

In [33]:
np.max(y[0, :, :], axis=0), np.min(y[0, :, :], axis=0)

np.max(y[0, :, 2:], axis=0) - np.min(y[0, :, :2], axis=0) - 512

array([1789.4011, 1789.4011], dtype=float32)

In [34]:
np.max(y[0, :, :2], axis=0) - np.min(y[0, :, 2:], axis=0) + 512

array([1793.3726, 1793.3726], dtype=float32)

In [4]:
from collections import namedtuple


In [27]:
temp = namedtuple('temp', ['var1', 'var2'])
temp.var1 = 'hello'
temp.var2 = 'world'

TypeError: descriptor 'index' requires a 'tuple' object but received a 'str'

In [1]:
import keras
import numpy as np


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [26]:
temp = np.array([[1,2,3,4],[5,6,7,8]])

temp[:, 1:3]

array([[2, 3],
       [6, 7]])